In [1]:
%matplotlib qt4
from __future__ import division

from models import tools, optimize, models, filters
from models.tests import PerformanceTest

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Initialization

In [2]:
data = tools.load_data(limit=200000, offset=1500000)
data = data[filters.sequentize(data) & filters.for_staircase(data)]
print len(data)

Loaded 176420 answers.
20984


## PFA

### Greedy Search

In [4]:
def pfa_fun(gamma, delta):
    elo = models.EloModel()
    pfa = models.PFAModel(prior=elo, gamma=gamma, delta=delta)
    pfa_test = PerformanceTest(pfa, landscape.data)

    pfa_test.run()
    return pfa_test.get_results().rmse

init_parameters = {'gamma': 2, 'delta': -1}
init_epsilons = {'gamma': 0.1, 'delta': 0.05}

landscape = optimize.GreedySearch(data)
landscape.search(pfa_fun, init_parameters, init_epsilons)

altitude: 0.377097440379
parameters: {'gamma': 0.9500315530032688, 'delta': -0.069594670956348553}
epsilons: {'gamma': 0.095170365465655521, 'delta': 0.0093821828364488279}


{'delta': -0.069594670956348553, 'gamma': 0.9500315530032688}

### Smart Greedy Search

In [28]:
landscape = optimize.GreedySearch2(data)
landscape.search_pfa(number_of_iter=10)

   0.82665   -0.11048    0.41463
   0.56001    0.15651    0.41306
   0.46866    0.28621    0.41304
   0.43680    0.33872    0.41342
   0.42541    0.35865    0.41364
   0.42129    0.36604    0.41373
   0.41978    0.36875    0.41376
   0.41924    0.36975    0.41377
   0.41904    0.37011    0.41378
   0.41897    0.37024    0.41378


{'delta': 0.37024055519326116, 'gamma': 0.41896607638479794}

## PFA/E

### Greedy Search

In [5]:
def pfae_fun(gamma, delta):
    elo = models.EloModel()
    pfae = models.PFAExt(elo, gamma=gamma, delta=delta)
    pfae_test = PerformanceTest(pfae, landscape.data)

    pfae_test.run()
    return pfae_test.get_results().rmse

init_parameters = {'gamma': 2, 'delta': -1}
init_epsilons = {'gamma': 0.1, 'delta': 0.05}

landscape = optimize.GreedySearch(data)
landscape.search(pfae_fun, init_parameters, init_epsilons)

altitude: 0.354064248528
parameters: {'gamma': 2.7659426003597627, 'delta': -0.76757226459496852}
epsilons: {'gamma': 0.079101073533933683, 'delta': 0.043112461933853298}


{'delta': -0.76757226459496852, 'gamma': 2.7659426003597627}

## PFA/G

### Greedy Search

In [7]:
def pfag_fun(gamma, delta, decay):
    elo = models.EloModel()
    pfag = models.PFAGong(prior=elo, gamma=gamma, delta=delta, decay=decay)
    pfag_test = PerformanceTest(pfag, landscape.data)

    pfag_test.run()
    return pfag_test.get_results().rmse

init_parameters = {'gamma': 2, 'delta': 0, 'decay': 0.5}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'decay': 0.02}

landscape = optimize.GreedySearch(data)
landscape.search(pfag_fun, init_parameters, init_epsilons)

altitude: 0.358844436988
parameters: {'delta': 0.14957220783956943, 'gamma': 1.5516771530480593, 'decay': 0.44460998218898112}
epsilons: {'delta': 0.044904869147786304, 'gamma': 0.068238198746089482, 'decay': 0.015099074035045167}


{'decay': 0.44460998218898112,
 'delta': 0.14957220783956943,
 'gamma': 1.5516771530480593}

### Smart Greedy Search

In [50]:
landscape = optimize.GreedySearch2(data)
landscape.search_pfag(number_of_iter=10)

   2.22006   -0.29863    0.37966
   1.85479    0.23149    0.37130
   1.58129    0.48080    0.36938
   1.40630    0.58719    0.36961
   1.29100    0.61561    0.37005
   1.21126    0.60626    0.37027
   1.15310    0.58130    0.37032
   1.10862    0.55186    0.37028
   1.07336    0.52300    0.37021
   1.04470    0.49676    0.37015


{'decay': 0.71608391039486985,
 'delta': 0.49676161027915278,
 'gamma': 1.044699976959059}

## PFA/E/T

### Greedy Search

In [4]:
def pfaet_fun(gamma, delta, a, c):
    elo = models.EloModel()
    pfaet = models.PFAExtTiming(
        elo, gamma=gamma, delta=delta,
        time_effect_fun=time_effect_fun, a=a, c=c)
    pfaet_test = PerformanceTest(pfaet, landscape.data)

    pfaet_test.run()
    return pfaet_test.get_results().rmse

In [5]:
time_effect_fun = 'pow'

init_parameters = {'gamma': 2, 'delta': -1, 'a': 3, 'c': 0.3}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'a': 0.1, 'c': 0.03}

landscape = optimize.GreedySearch(data)
landscape.search(pfaet_fun, init_parameters, init_epsilons, maxiter=30)

altitude: 0.374310118189
parameters: {'a': 3.0954052130870449, 'c': 0.21320363243987012, 'gamma': 2.0930829813020249, 'delta': -0.90359497562913749}
epsilons: {'a': 0.092225242009501288, 'c': 0.02436023626270311, 'gamma': 0.086235526334246249, 'delta': 0.043386666250166565}


{'a': 3.0954052130870449,
 'c': 0.21320363243987012,
 'delta': -0.90359497562913749,
 'gamma': 2.0930829813020249}

In [6]:
time_effect_fun = 'log'

init_parameters = {'gamma': 2, 'delta': -1, 'a': 1.5, 'c': 0.1}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'a': 0.1, 'c': 0.03}

landscape = optimize.GreedySearch(data)
landscape.search(pfaet_fun, init_parameters, init_epsilons, maxiter=30)

altitude: 0.374254119587
parameters: {'a': 1.513257494926556, 'c': 0.096867480621462942, 'gamma': 2.0018347628219155, 'delta': -0.99486886871371072}
epsilons: {'a': 0.072005921572274714, 'c': 0.019397633497346498, 'gamma': 0.080066196854110261, 'delta': 0.041701409218529273}


{'a': 1.513257494926556,
 'c': 0.096867480621462942,
 'delta': -0.99486886871371072,
 'gamma': 2.0018347628219155}

In [7]:
time_effect_fun = 'exp'

init_parameters = {'gamma': 2, 'delta': -1, 'a': 1, 'c': 0.01}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'a': 0.1, 'c': 0.005}

landscape = optimize.GreedySearch(data)
landscape.search(pfaet_fun, init_parameters, init_epsilons, maxiter=30)

altitude: 0.374629437147
parameters: {'a': 1.1, 'c': 0.005, 'gamma': 2.1, 'delta': -0.95}
epsilons: {'a': 0.099385335904517619, 'c': 0.004851883029818811, 'gamma': 0.098644526514727798, 'delta': 0.049232804890392237}


{'a': 1.1, 'c': 0.005, 'delta': -0.95, 'gamma': 2.1}

## PFA/G/T

### Greedy Search

In [8]:
def pfagt_fun(a, c, b, d):
    elo = models.EloModel()
    pfagt = models.PFATiming(
        elo,
        time_effect_good=time_effect_fun, a=a, c=c,
        time_effect_bad=time_effect_fun, b=b, d=d)
    pfagt_test = PerformanceTest(pfagt, landscape.data)

    pfagt_test.run()
    return pfagt_test.get_results().rmse

In [9]:
time_effect_fun = 'pow'

init_parameters = {'a': 3, 'c': 0.3, 'b': 4, 'd': 0.4}
init_epsilons = {'a': 0.1, 'c': 0.03, 'b': 0.1, 'd': 0.04}

landscape = optimize.GreedySearch(data)
landscape.search(pfagt_fun, init_parameters, init_epsilons, maxiter=30)

altitude: 0.391614242522
parameters: {'a': 3.2854110952418063, 'c': 0.22628209829830848, 'b': 4.6779139573229163, 'd': 0.93519366624420974}
epsilons: {'a': 0.088252799215740349, 'c': 0.012463364504304589, 'b': 0.097818099072375944, 'd': 0.034527914820945922}


{'a': 3.2854110952418063,
 'b': 4.6779139573229163,
 'c': 0.22628209829830848,
 'd': 0.93519366624420974}

In [10]:
time_effect_fun = 'log'

init_parameters = {'a': 1.7, 'c': 0.1, 'b': 0.9, 'd': 0.1}
init_epsilons = {'a': 0.05, 'c': 0.005, 'b': 0.05, 'd': 0.005}

landscape = optimize.GreedySearch(data)
landscape.search(pfagt_fun, init_parameters, init_epsilons, maxiter=30)

altitude: 0.393678983702
parameters: {'a': 1.6866305227760314, 'c': 0.10088571006549757, 'b': 0.89091817277688767, 'd': 0.095267255973747966}
epsilons: {'a': 0.027481914572241684, 'c': 0.0025967693241263712, 'b': 0.031969386990870416, 'd': 0.0029325600994641638}


{'a': 1.6866305227760314,
 'b': 0.89091817277688767,
 'c': 0.10088571006549757,
 'd': 0.095267255973747966}

In [11]:
time_effect_fun = 'exp'

init_parameters = {'a': 1.5, 'c': 0.01, 'b': 1.3, 'd': 0.01}
init_epsilons = {'a': 0.1, 'c': 0.003, 'b': 0.1, 'd': 0.003}

landscape = optimize.GreedySearch(data)
landscape.search(pfagt_fun, init_parameters, init_epsilons, maxiter=30)

altitude: 0.393682259936
parameters: {'a': 0.72692085941193285, 'c': 0.0017535316706194378, 'b': 0.737691772545225, 'd': 0.088884542394713134}
epsilons: {'a': 0.078297035674251475, 'c': 0.0010110572022579186, 'b': 0.078811036407209364, 'd': 0.0025582988105892438}


{'a': 0.72692085941193285,
 'b': 0.737691772545225,
 'c': 0.0017535316706194378,
 'd': 0.088884542394713134}

### Smart Greedy Search

In [49]:
landscape = optimize.GreedySearch2(data)
landscape.search_pfagt(number_of_iter=10)

   3.51309   -0.25624    0.37776
   3.68964    0.04912    0.37706
   3.46514    0.04062    0.37575
   3.25846    0.03920    0.37536
   3.05692    0.03611    0.37501
   2.86231    0.03415    0.37464
   2.67456    0.03202    0.37430
   2.49209    0.02844    0.37398
   2.32170    0.02788    0.37366
   2.15652    0.02566    0.37334


{'a': 2.9700846520609057,
 'c': 0.25924650629428969,
 'delta': 0.025657149029072922,
 'gamma': 2.1565236199060083}

In [3]:
landscape = optimize.GreedySearch2(data)
landscape.search_pfagt(number_of_iter=10, time_effect_fun='exp')

   2.01599    0.10362    0.38777
   1.68531    0.05819    0.38661
   1.34697    0.03985    0.38363
   1.06414    0.03902    0.38051
   0.79137    0.02831    0.37611
   0.55974    0.01953    0.37206
   0.34258    0.02003    0.36915
   0.20341    0.04402    0.37436
   0.11545    0.02245    0.38869
   0.08667    0.01565    0.39558


{'a': 3.2731618029154732,
 'c': -0.099687077060505391,
 'delta': 0.015650741856747617,
 'gamma': 0.086671413614965359}

## PFA/E/RT

In [12]:
def pfart_fun(gamma, delta, zeta):
    elo = models.EloModel()
    pfart = models.PFAResponseTime(elo, gamma=gamma, delta=delta, zeta=zeta)
    pfart_test = PerformanceTest(pfart, landscape.data)

    pfart_test.run()
    return pfart_test.get_results().rmse

init_parameters = {'gamma': 2, 'delta': -1, 'zeta': 1.5}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'zeta': 0.1}

landscape = optimize.GreedySearch(data)
landscape.search(pfart_fun, init_parameters, init_epsilons, maxiter=30)

altitude: 0.379161215004
parameters: {'zeta': 3.4169393094296372, 'gamma': 1.8593913082626701, 'delta': -1.2234781847569738}
epsilons: {'zeta': 0.060922324942620218, 'gamma': 0.082591137048408148, 'delta': 0.035411705703641612}


{'delta': -1.2234781847569738,
 'gamma': 1.8593913082626701,
 'zeta': 3.4169393094296372}